traverse_marginal, marginal_trees

marginal_trees -> (get_breakpoints, marginal_arg -> traverse_marginal)

I think i need to:

- Add interval as argument to marginal_trees
#- Only include breakpoints where both recombination parents are in the marginal arg
- Add flanking borders to breakpoints that 

In [11]:
# from popgen_dashboards.arg_dashboard import app
# app.run_server(mode='inline', height=900, debug=True)

from arg_dashboard import app
import arg_dashboard

import arg_dashboard
from importlib import reload
reload(arg_dashboard)

arg_dashboard.app.run_server(mode='inline', height=900, debug=True)

In [30]:
def segments_crossing(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return False
    #    raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div

    # return x, y

    return \
        min(line1[0][0], line1[1][0]) < x < max(line1[0][0], line1[1][0]) and \
        min(line2[0][0], line2[1][0]) < x < max(line2[0][0], line2[1][0]) and \
        min(line1[0][1], line1[1][1]) < y < max(line1[0][1], line1[1][1]) and \
        min(line2[0][1], line2[1][1]) < y < max(line2[0][1], line2[1][1])


def crossing(a, b):
    return segments_crossing(
        ((a.down.xpos, a.down.height), (a.up.xpos, a.up.height)), 
        ((b.down.xpos, b.down.height), (b.up.xpos, b.up.height))
        )

def get_all_crossing_pairs(lineages):
    crossing_pairs = []
    for i in range(len(lineages)):
        for j in range(i+1, len(lineages)):
            if crossing(lineages[i], lineages[j]):
                crossing_pairs.append([i, j])
    return crossing_pairs

def improve_x_positions(nodes):

    lineages = get_parent_lineages(nodes)
    crossing_pairs = get_all_crossing_pairs(lineages)

    for i, j in crossing_pairs:
        # swap ups
        lineages[i].up.xpos, lineages[j].up.xpos = lineages[j].up.xpos, lineages[i].up.xpos
        new_crossing_pairs = get_all_crossing_pairs(lineages)
        if not len(new_crossing_pairs) < len(crossing_pairs):
            # swap ups back
            lineages[i].up.xpos, lineages[j].up.xpos = lineages[j].up.xpos, lineages[i].up.xpos

            # swap downs
            lineages[i].down.xpos, lineages[j].down.xpos = lineages[j].down.xpos, lineages[i].down.xpos
            new_crossing_pairs = get_all_crossing_pairs(lineages)
            if not len(new_crossing_pairs) < len(crossing_pairs):
                # swap downs back
                lineages[i].down.xpos, lineages[j].down.xpos = lineages[j].down.xpos, lineages[i].down.xpos


from arg import Lineage, Coalescent
import numpy as np
a = Lineage(down=Coalescent(xpos=1, height=10), up=Coalescent(xpos=20, height=12))
b = Lineage(down=Coalescent(xpos=0, height=0), up=Coalescent(xpos=20, height=20))

crossing(a, b)
get_all_crossing_pairs([a, b])

[[0, 1]]

In [31]:
# def line_intersection(line1, line2):
#     xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
#     ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

#     def det(a, b):
#         return a[0] * b[1] - a[1] * b[0]

#     div = det(xdiff, ydiff)
#     if div == 0:
#        raise Exception('lines do not intersect')

#     d = (det(*line1), det(*line2))
#     x = det(d, xdiff) / div
#     y = det(d, ydiff) / div
#     return x, y

# def crossing(line1, line2):
#     xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
#     ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

#     def det(a, b):
#         return a[0] * b[1] - a[1] * b[0]

#     div = det(xdiff, ydiff)
#     if div == 0:
#         return False
#     #    raise Exception('lines do not intersect')

#     d = (det(*line1), det(*line2))
#     x = det(d, xdiff) / div
#     y = det(d, ydiff) / div

#     print(x, y)
#     # return x, y

#     return \
#         min(line1[0][0], line1[1][0]) < x < max(line1[0][0], line1[1][0]) and \
#         min(line2[0][0], line2[1][0]) < x < max(line2[0][0], line2[1][0]) and \
#         min(line1[0][1], line1[1][1]) < y < max(line1[0][1], line1[1][1]) and \
#         min(line2[0][1], line2[1][1]) < y < max(line2[0][1], line2[1][1])



# # print(line_intersection((A, B), (C, D)))
# print(line_intersection(((0, 10), (20, 12)), ((0, 0), (20, 20))))